This notebook is based on the tensorflow docs: 


https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams

adapted for a regression problem.

In [2]:
%load_ext tensorboard
# Clear any logs from previous runs
#!del -rf "./logs/"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler

In [4]:
a = tuple(range(1, 6))
type(a)
a

(1, 2, 3, 4, 5)

In [5]:
file = "https://raw.githubusercontent.com/la-escamilla/codigos/master/rho_tanh15_phy_rho_corr%2BCBAO%2BHD%2BSN_nested_dynesty_multi_1.txt"

split = 0.8
epochs = 500

cols = tuple(range(1, 19))
data = np.loadtxt(file, usecols=cols)
params = data[:, 1:]
logL = data[:, 0]


nparams = len(params)
randomize = np.random.permutation(nparams)
params = params[randomize]
logL = logL[randomize]

ntrain = int(split * nparams)
indx = [ntrain]

params_training, params_testing = np.split(params, indx)
logL_training, logL_testing = np.split(logL, indx)


# # create scaler
# scaler = StandardScaler()
# # # scaler = MinMaxScaler(feature_range=(-1,1))
# # # fit scaler on data
# scaler.fit(params_training)
# # # apply transform
# params_training = scaler.transform(params_training)
# params_testing = scaler.transform(params_testing)

print("Total len dataset {}," 
      "len training set {}, len test set {}".format(nparams, 
                                                 len(params_training), len(params_testing)))



Total len dataset 40039,len training set 32031, len test set 8008


In [6]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=0,
                                   patience=10,
                                   restore_best_weights=True)
                                   ]

n_cols = params_training.shape[1]

In [7]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([100, 150, 200]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([100, 150, 200]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([100, 150, 200]))

HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([128]))

In [8]:
# METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
      hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])


In [9]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS1], activation=tf.nn.relu, input_shape=(n_cols,)),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS2], activation=tf.nn.relu),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS3], activation=tf.nn.relu),
        tf.keras.layers.Dense(1)
    ])
                              
    model.compile(
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )

    
    model.fit(params_training, logL_training, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], callbacks=callbacks) 
    _, loss = model.evaluate(params_testing, logL_testing)


    return loss
    

In [10]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [ ]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
        for num_units2 in HP_NUM_UNITS2.domain.values:
            for num_units3 in HP_NUM_UNITS3.domain.values:
                hparams = {
                    HP_NUM_UNITS1: num_units1,
                    HP_NUM_UNITS2: num_units2,
                    HP_NUM_UNITS3: num_units3,
                    HP_BATCHSIZE: batch_size
                }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('logs/hparam_tuning/' + run_name, hparams)
                session_num += 1

                    

--- Starting trial: run-0
{'num_units1': 100, 'num_units2': 100, 'num_units3': 100, 'batch_size': 128}
Train on 32031 samples
Epoch 1/500
32031/32031 [==============================] - 1s 20us/sample - loss: 873969.8343 - mean_squared_error: 873970.0000
Epoch 2/500
32031/32031 [==============================] - 0s 10us/sample - loss: 418152.4864 - mean_squared_error: 418152.7188
Epoch 3/500
32031/32031 [==============================] - 0s 10us/sample - loss: 151153.9687 - mean_squared_error: 151153.9844
Epoch 4/500
32031/32031 [==============================] - 0s 10us/sample - loss: 97439.0658 - mean_squared_error: 97439.0859
Epoch 5/500
32031/32031 [==============================] - 0s 10us/sample - loss: 63155.8784 - mean_squared_error: 63155.8594
Epoch 6/500
32031/32031 [==============================] - 0s 10us/sample - loss: 38245.4992 - mean_squared_error: 38245.4883
Epoch 7/500
32031/32031 [==============================] - 0s 10us/sample - loss: 23951.7431 - mean_squared_erro

32031/32031 [==============================] - 0s 10us/sample - loss: 3613.4420 - mean_squared_error: 3613.4414
Epoch 66/500
32031/32031 [==============================] - 0s 10us/sample - loss: 3550.2981 - mean_squared_error: 3550.2981
Epoch 67/500
32031/32031 [==============================] - 0s 10us/sample - loss: 3603.8506 - mean_squared_error: 3603.8511
Epoch 68/500
32031/32031 [==============================] - 0s 10us/sample - loss: 3522.9122 - mean_squared_error: 3522.9106
Epoch 69/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3661.8482 - mean_squared_error: 3661.8484
Epoch 70/500
32031/32031 [==============================] - 0s 10us/sample - loss: 3501.9761 - mean_squared_error: 3501.9746
Epoch 71/500
32031/32031 [==============================] - 0s 10us/sample - loss: 3482.5859 - mean_squared_error: 3482.5862
Epoch 72/500
32031/32031 [==============================] - 0s 10us/sample - loss: 3371.4071 - mean_squared_error: 3371.4077
Epoch 73/500


32031/32031 [==============================] - 0s 10us/sample - loss: 2689.5988 - mean_squared_error: 2689.5989
Epoch 130/500
32031/32031 [==============================] - 0s 10us/sample - loss: 2678.2841 - mean_squared_error: 2678.2832
Epoch 131/500
32031/32031 [==============================] - 0s 10us/sample - loss: 2906.7862 - mean_squared_error: 2906.7861
Epoch 132/500
32031/32031 [==============================] - 0s 10us/sample - loss: 2619.9170 - mean_squared_error: 2619.9167
Epoch 133/500
32031/32031 [==============================] - 0s 10us/sample - loss: 2869.3611 - mean_squared_error: 2869.3616
Epoch 134/500
32031/32031 [==============================] - 0s 10us/sample - loss: 2693.6207 - mean_squared_error: 2693.6208
Epoch 135/500
32031/32031 [==============================] - 0s 10us/sample - loss: 2769.5435 - mean_squared_error: 2769.5430
Epoch 136/500
32031/32031 [==============================] - 0s 9us/sample - loss: 2695.0044 - mean_squared_error: 2695.0049
Epoch 1

32031/32031 [==============================] - 0s 11us/sample - loss: 4261.7086 - mean_squared_error: 4261.7100
Epoch 45/500
32031/32031 [==============================] - 0s 11us/sample - loss: 4415.7314 - mean_squared_error: 4415.7310
Epoch 46/500
32031/32031 [==============================] - 0s 14us/sample - loss: 4315.0058 - mean_squared_error: 4315.0054
Epoch 47/500
32031/32031 [==============================] - 0s 14us/sample - loss: 4114.5587 - mean_squared_error: 4114.5586
Epoch 48/500
32031/32031 [==============================] - 0s 13us/sample - loss: 4137.5178 - mean_squared_error: 4137.5176
Epoch 49/500
32031/32031 [==============================] - 0s 12us/sample - loss: 4385.8017 - mean_squared_error: 4385.8032
Epoch 50/500
32031/32031 [==============================] - 0s 11us/sample - loss: 3927.7568 - mean_squared_error: 3927.7571
Epoch 51/500
32031/32031 [==============================] - 0s 11us/sample - loss: 4103.1474 - mean_squared_error: 4103.1470
Epoch 52/500


32031/32031 [==============================] - 0s 11us/sample - loss: 2958.4193 - mean_squared_error: 2958.4187
Epoch 110/500
32031/32031 [==============================] - 0s 11us/sample - loss: 2819.5411 - mean_squared_error: 2819.5403
Epoch 111/500
32031/32031 [==============================] - 0s 11us/sample - loss: 2933.6995 - mean_squared_error: 2933.6995
Epoch 112/500
32031/32031 [==============================] - 0s 11us/sample - loss: 3104.1697 - mean_squared_error: 3104.1702
Epoch 113/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3025.0923 - mean_squared_error: 3025.0923
Epoch 114/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3136.9305 - mean_squared_error: 3136.9299
Epoch 115/500
32031/32031 [==============================] - 0s 12us/sample - loss: 2855.3206 - mean_squared_error: 2855.3206
Epoch 116/500
32031/32031 [==============================] - 0s 12us/sample - loss: 2789.1976 - mean_squared_error: 2789.1980
Epoch 

...

32031/32031 [==============================] - 1s 16us/sample - loss: 3741.3791 - mean_squared_error: 3741.3789
Epoch 50/500
32031/32031 [==============================] - 0s 15us/sample - loss: 3537.3741 - mean_squared_error: 3537.3735
Epoch 51/500
32031/32031 [==============================] - 0s 15us/sample - loss: 3505.0518 - mean_squared_error: 3505.0522
Epoch 52/500
32031/32031 [==============================] - 0s 15us/sample - loss: 3690.1601 - mean_squared_error: 3690.1619
Epoch 53/500
32031/32031 [==============================] - 0s 13us/sample - loss: 3366.2491 - mean_squared_error: 3366.2478
Epoch 54/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3317.9708 - mean_squared_error: 3317.9717
Epoch 55/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3286.6723 - mean_squared_error: 3286.6733
Epoch 56/500
32031/32031 [==============================] - 1s 17us/sample - loss: 3252.2722 - mean_squared_error: 3252.2720
Epoch 57/500


32031/32031 [==============================] - 0s 13us/sample - loss: 5494.5100 - mean_squared_error: 5494.5117
Epoch 21/500
32031/32031 [==============================] - 0s 14us/sample - loss: 5367.3083 - mean_squared_error: 5367.3086
Epoch 22/500
32031/32031 [==============================] - 0s 14us/sample - loss: 5312.9773 - mean_squared_error: 5312.9785
Epoch 23/500
32031/32031 [==============================] - 0s 13us/sample - loss: 4968.0602 - mean_squared_error: 4968.0586
Epoch 24/500
32031/32031 [==============================] - 0s 13us/sample - loss: 5058.2967 - mean_squared_error: 5058.2974
Epoch 25/500
32031/32031 [==============================] - 0s 13us/sample - loss: 5197.1376 - mean_squared_error: 5197.1372
Epoch 26/500
32031/32031 [==============================] - 0s 13us/sample - loss: 4692.2717 - mean_squared_error: 4692.2710
Epoch 27/500
32031/32031 [==============================] - 0s 13us/sample - loss: 4659.1216 - mean_squared_error: 4659.1221
Epoch 28/500


32031/32031 [==============================] - 1s 17us/sample - loss: 2769.1178 - mean_squared_error: 2769.1162
Epoch 86/500
32031/32031 [==============================] - 0s 14us/sample - loss: 2637.4019 - mean_squared_error: 2637.4019
Epoch 87/500
32031/32031 [==============================] - 0s 14us/sample - loss: 2730.7824 - mean_squared_error: 2730.7825
Epoch 88/500
32031/32031 [==============================] - 0s 14us/sample - loss: 2705.0115 - mean_squared_error: 2705.0120
Epoch 89/500
32031/32031 [==============================] - 0s 14us/sample - loss: 2732.5911 - mean_squared_error: 2732.5916
Epoch 90/500
32031/32031 [==============================] - 0s 14us/sample - loss: 2640.0392 - mean_squared_error: 2640.0386
Epoch 91/500
32031/32031 [==============================] - 0s 15us/sample - loss: 2785.5987 - mean_squared_error: 2785.5994
Epoch 92/500
32031/32031 [==============================] - 0s 15us/sample - loss: 2828.8686 - mean_squared_error: 2828.8694
Epoch 93/500


32031/32031 [==============================] - 0s 13us/sample - loss: 3719.7526 - mean_squared_error: 3719.7529
Epoch 52/500
32031/32031 [==============================] - 0s 13us/sample - loss: 3595.4028 - mean_squared_error: 3595.4036
Epoch 53/500
32031/32031 [==============================] - 0s 13us/sample - loss: 3664.8750 - mean_squared_error: 3664.8762
Epoch 54/500
32031/32031 [==============================] - 0s 13us/sample - loss: 3706.4098 - mean_squared_error: 3706.4099
Epoch 55/500
32031/32031 [==============================] - 0s 13us/sample - loss: 3378.2945 - mean_squared_error: 3378.2947
Epoch 56/500
32031/32031 [==============================] - 0s 13us/sample - loss: 3678.3032 - mean_squared_error: 3678.3027
Epoch 57/500
32031/32031 [==============================] - 0s 13us/sample - loss: 3455.0659 - mean_squared_error: 3455.0652
Epoch 58/500
32031/32031 [==============================] - 0s 13us/sample - loss: 3663.5943 - mean_squared_error: 3663.5957
Epoch 59/500


32031/32031 [==============================] - 0s 13us/sample - loss: 2519.6245 - mean_squared_error: 2519.6240
Epoch 117/500
32031/32031 [==============================] - 0s 14us/sample - loss: 2647.6545 - mean_squared_error: 2647.6550
Epoch 118/500
32031/32031 [==============================] - 0s 13us/sample - loss: 2553.0050 - mean_squared_error: 2553.0051
Epoch 119/500
32031/32031 [==============================] - 0s 13us/sample - loss: 2445.5435 - mean_squared_error: 2445.5439
Epoch 120/500
32031/32031 [==============================] - 0s 13us/sample - loss: 2476.9168 - mean_squared_error: 2476.9172
Epoch 121/500
32031/32031 [==============================] - 0s 13us/sample - loss: 2571.6532 - mean_squared_error: 2571.6536
Epoch 122/500
32031/32031 [==============================] - 0s 15us/sample - loss: 2636.7801 - mean_squared_error: 2636.7800
Epoch 123/500
32031/32031 [==============================] - 0s 14us/sample - loss: 2678.7181 - mean_squared_error: 2678.7168
Epoch 

In [30]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 17368), started 0:03:22 ago. (Use '!kill 17368' to kill it.)